# Example Notebook illustrating the use of Olex2 in QCrBox

## Import and setting up folders / paths

In [ ]:
import shutil

from qcrbox_wrapper import QCrBoxPathHelper, QCrBoxWrapper

We create a path to run our Olex2 examples and get our paths from our own computer's filesystem and the internal filesystem of the QCrBox containers.

Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, use the `__init__` method instead by defining the path to the shared directory explicitely in `path_to_shared_dir` and replacing the next four lines with:

```python
pathhelper = QCrBoxPathHelper(
    path_to_shared_dir,
    'examples_olex2'
)
```

In [ ]:
pathhelper = QCrBoxPathHelper.from_dotenv(".env.dev", "examples_olex2")

We now have two python pathlib objects we can use to make the reference to our pathes much easier. We have also created an examples_olex2 folder we will use to work for this notebook.

In [ ]:
path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

Let us first connect to the QCrBox. Make sure that you have actually started up the container with `qcb up` in the terminal. Sometimes it can take a while for the Server to come online, so you might need to retry a few times. 

In [ ]:
qcrbox = QCrBoxWrapper.from_server_addr("127.0.0.1", 11000)

We can now see what applications are currently available and how we can access them. Olex2 (Linux) should show up here and therefore we select it in the new cell.

In [ ]:
qcrbox.application_dict

In [ ]:
olex2 = qcrbox.application_dict["Olex2 (Linux)"]

Using the python buildin help function we can see which commands are available for the given container.

In [ ]:
help(olex2)

We can now run our first command. QCrBox works with a subsect of cif entries, which will be called *unified cif entries* in this explanation. The subset are the base cif entries in the current cif dictionaries (as of March 2024). The conversion of these entries into a format the individual programs can understand is programmed into the commands themselves. This way the aliases of the cif format do not need to be taken into account by the developers of the software. If you want to create such a cif file from any cif, look at the `qcrboxtools.to_unified_cif function` or the QCrBoxTools library.

## Running an IAM refinement

For the time being we copy our example cif file into the folder we will work in. We can then use the `olex2.refine_iam` function to do a refinement against spherical atomic form factors. Notice that all our python code uses the local path, while the function runs in the container itself and therefore needs a path relative to the container (derived from `path_qcrbox`).

Finally, all functions in containers are executed asynchronously and non-blocking in our script. In order to wait until the function has been executed we can use the `wait_while_running` function of the returned calculated object

In [ ]:
folder_iam = path_local / "run_refine_iam"
folder_iam.mkdir(exist_ok=True)

shutil.copy("./input_files/input.cif", folder_iam / "input.cif")

calc = olex2.refine_iam(
    input_cif_path=path_qcrbox / "run_refine_iam" / "input.cif",
    output_cif_path=path_qcrbox / "run_refine_iam" / "output.cif",
    ls_cycles="20",
    weight_cycles="5",
)

calc.wait_while_running(1.0)

What will happen under the hood is that qcrbox will generate a `work.cif` that has all the entries in aliases Olex2 understands. It will then run the refinement. Afterwards it will create a new cif that uses unified keywords again called `output.cif`.

Finally we can have a look at that calculation object. It gives updated information about the calculation and in many cases information why a calculation might have failed. Fortunately, our status should now show 'completed', which means everything went as intended.

In [ ]:
calc.status

If you have a failure in the execution of a command it is always a good idea to look into the log of your docker container. The way Olex2 is executed here gives an additional source of information: `task_*.log` files. Have a look in your folder, where several of these should now be located.

# Running a refinement with a tsc file.

There is also exposed functionality to run the refinement with a `tsc(b)` file. Let's do just that

In [ ]:
folder_tsc = path_local / "run_refine_tsc"
folder_tsc.mkdir(exist_ok=True)

shutil.copy("./input_files/input.cif", folder_tsc / "input.cif")
shutil.copy("./input_files/example.tscb", folder_tsc / "example.tscb")

calc = olex2.refine_tsc(
    input_cif_path=path_qcrbox / "run_refine_tsc" / "input.cif",
    output_cif_path=path_qcrbox / "run_refine_iam" / "output.cif",
    tsc_path=path_qcrbox / "run_refine_tsc" / "example.tscb",
    ls_cycles="20",
    weight_cycles="5",
)

calc.wait_while_running(1.0)

In [ ]:
calc.status

If you go to the folder, you'll see that there are again our cif files. The task_*.log file with the highest number should contain the progress of your refinement. If you want to, you can compare the numbers to what was obtained by the IAM refinement.

## Running arbitrary Olex console commands.

Finally, there is the option to run arbitrary olex2 console commands from a file. Let us put everything in a folder first

In [ ]:
folder_cmd = path_local / "run_cmd_file"
folder_cmd.mkdir(exist_ok=True)

shutil.copy("./input_files/input.cif", folder_cmd / "input.cif");

Now we create a text file that includes the commands we want to execute as lines. Let's create a `structure.xyz` file from our cif. 

In [ ]:
commands = ["file structure.xyz"]

command_file = folder_cmd / "cmd.input"
command_file.write_text("\n".join(commands))

calc = olex2.run_cmds_file(
    input_cif_path=path_qcrbox / "run_cmd_file" / "input.cif",
    output_cif_path=path_qcrbox / "run_cmd_file" / "output.cif",
    cmd_file_path=path_qcrbox / "run_cmd_file" / "cmd.input",
)

calc.wait_while_running(0.5)

In [ ]:
calc.status

You might see that there is no `output.cif` file in the folder. The `output.cif` is only generated if the work.cif has been changed during our operations. If you want to, you can use a command like 'refine 10' to actually change the `work.cif` and check that the `output.cif` file is generated in this case.

## Running the Olex2 GUI from QCrBox

While the other commands are run non-interactively, we can also run the Olex2 GUI from QCrBox. The exposed command for this purpose is called `interactive`. However, we do face a problem. Olex2 works and outputs other cif entries than unified ones, but we only want to convert once we are done with whatever we want to to in the GUI. 

Once we run the `interactive` command, two things happen: Firstly, a new browser window with the GUI opens up. Secondly, you should see an input prompt in your execution engine for this notebook. (In Jupyter this should be at the end of the cell, in VSCode it is at the top). By pressing enter within this prompt we tell QCrBox that we are done and the newest cif in the folder is supposed to be converted. Note that this is a stopgap for an actual UI, so this feels a bit clunky at the moment.

Once you have done something in the UI and have pressed enter you should see an `output.cif` in the `run_interactive` folder.

In [ ]:
folder_interactive = path_local / "run_interactive"
folder_interactive.mkdir(exist_ok=True)

input_interactive = folder_interactive / "input.cif"
shutil.copy("./input_files/input.cif", input_interactive)

session = olex2.interactive_session(
    input_cif_path=path_qcrbox / "run_interactive" / "input.cif",
    work_cif_path=path_qcrbox / "run_interactive" / "work.cif",
    output_cif_path=path_qcrbox / "run_interactive" / "output.cif",
    parameter_json_path=path_qcrbox / "run_interactive" / "parameters.json",
    parameter_folder=path_qcrbox / "run_interactive" / "parameters",
)
session.start()

## Final Remarks
This concludes this notebook explaining how to interact with the Olex2 container. You might realise that there are to functions `to_params__interactive` and `redo__interactive`. These will be explained in the developer documentation. 